In [16]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchase File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Use the head() function to display column headers and the first 5 rows
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count
 * Display the total number of players

In [17]:
# Use the screen name column (SN) to find the number of unique players. Count would not work because players could make more than one purchase
# Use len() to get the length of the list
total_unique_players = len(purchase_data["SN"].unique())

# Create a new dataframe to display the number of total players
total_players_df = pd.DataFrame([total_unique_players], columns = ["Total Players"])
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)
 * Run basic calculations to obtain number of: 
    * Number of Unique Items
    * Average Purchase Price
    * Total Number of Purchases
    * Total Revenue

* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [18]:
# Use the item name column (Item Name) to find the number of unique items with the unique() function
# Use len() to get the length of the list
unique_items = len(purchase_data["Item Name"].unique())

# Use the count() function to get the total number of purchases
number_of_purchases = purchase_data["Price"].count()

# Calculate the total_revenue by using the sum() function on the price column (Price)
total_revenue = purchase_data["Price"].sum()

# Calculate the average purchase price by dividing total_revenue by number_of_purchases
average_purchase_price = purchase_data["Price"].mean()

# Create a data frame to display findings
purchase_summary_df = pd.DataFrame({"Number of Unique Items" : [unique_items],
                                    "Average Price" : average_purchase_price,
                                    "Number of Purchases" : number_of_purchases,
                                    "Total Revenue" : total_revenue})

# Format the data for cleaner appearance
purchase_summary_df["Average Price"] = purchase_summary_df["Average Price"].map("${:,.2f}".format)
purchase_summary_df["Total Revenue"] = purchase_summary_df["Total Revenue"].map("${:,.2f}".format)

# Display the summary table
purchase_summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed


In [19]:
# First create a data frame that is groupby() gender
gender_group_df = purchase_data.groupby("Gender")

# Find the number of unique screen names in each group of gender by using nunique(), whereas unique() would provide lists instead of numbers of unique entries per group
unique_gender_group = gender_group_df["SN"].nunique()

# Calculate the percentage of each group by dividing by the total number of unique players we calculated earlier
gender_percentage = unique_gender_group / total_unique_players * 100

# Create a data frame to display findings
gender_summary_df = pd.DataFrame({"Total Count" : unique_gender_group,
                                  "Percentage of Players" : gender_percentage})

# Format the data for cleaner appearance
gender_summary_df["Percentage of Players"] = gender_summary_df["Percentage of Players"].map("{:.2f}%".format)

# Remove the name of the index
gender_summary_df.index.name = None

# Sort the values for the data frame in descending order based on percentage of players
gender_summary_df = gender_summary_df.sort_values("Percentage of Players", ascending = False)

# Display the summary table
gender_summary_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain the following by Gender:

    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value
    * Average Purchase Total per Person

* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [20]:
# Calculate the purchase count by gender by using the count() function on the previously created gender_group_df
gender_purchase_count = gender_group_df["Item Name"].count()

# Calculate the total purchase value by gender by using the sum() function
gender_purchase_total = gender_group_df["Price"].sum()

# Calculate the average purchase price by gender
average_gender_purchase_price = gender_group_df["Price"].mean()

# Calculate the average total purchase value per person by gender
average_gender_total_purchase = gender_purchase_total / unique_gender_group

# Create a data frame to display findings
gender_purchase_summary_df = pd.DataFrame({"Purchase Count" : gender_purchase_count,
                                           "Average Purchase Price" : average_gender_purchase_price,
                                           "Total Purchase Value": gender_purchase_total,
                                           "Average Total Purchase per Person" : average_gender_total_purchase})

# Format the data for cleaner appearance
# Use iloc() to select all the rows for columns that need to be formatted, then applymap() to format the entire selection
gender_purchase_summary_df.iloc[ :, 1: ] = gender_purchase_summary_df.iloc[ : , 1:].applymap("${:,.2f}".format)

# Add the index name
gender_purchase_summary_df.index.name = "Gender"

# Display the summary table
gender_purchase_summary_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages

    * Break down into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) and calculate the following by age group:

        * Purchase Count
        * Average Purchase Price
        * Total Purchase Value
        * Average Purchase Total per Person

* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table

In [21]:
# Create bins of 4 years
# By using purchase_data["Age"].min() and purchase_data["Age"].max() we can see the lowest and highest values to help us set the bins lower and upper bounds
age_bins = [5, 9, 14, 19, 24, 29, 34, 39, 50]

# Create labels for the groups
age_group_labels = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Create a copy of the purchase_data data frame so we can keep a separate data frame to work with
# Use pd.cut() to categorize the ages of players into the group labels and add them to a new column
age_df = purchase_data.copy()
age_df["Age Groups"] = pd.cut(age_df["Age"], age_bins, labels = age_group_labels)

# Group the dataframe by the new Age Group column
# Find the number of unique screen names in each group
age_group = age_df.groupby("Age Groups")
age_group_unique = age_group["SN"].nunique()

# Calculate the percentage of each group of players from total unique players
age_group_percentage = age_group_unique / total_unique_players * 100

# Create a summary table to display findings
age_group_summary_df = pd.DataFrame({"Total Count" : age_group_unique,
                                     "Percentage of Players" : age_group_percentage})

# Remove the name of the index
age_group_summary_df.index.name = None

# Format the data for cleaner appearance
age_group_summary_df["Percentage of Players"] = age_group_summary_df["Percentage of Players"].map("{:.2f}%".format)

# Display the summary table
age_group_summary_df

,Total Count,Percentage of Players
< 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain the following by age group:

    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value
    * Average Purchase Total per Person


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [22]:
# Calculate the purchase count by age group by using the count() function on the previously created age_group
age_purchase_count = age_group["Item Name"].count()

# Calculate the total purchase value by age group by using the sum() function
age_purchase_total = age_group["Price"].sum()

# Calculate the average purchase price by age group
average_age_purchase_price = age_group["Price"].mean()

# Calculate the average total purchase value per person by age group
average_age_total_purchase = age_purchase_total / age_group_unique

# Create a data frame to display findings
age_purchase_summary_df = pd.DataFrame({"Purchase Count" : age_purchase_count,
                                           "Average Purchase Price" : average_age_purchase_price,
                                           "Total Purchase Value": age_purchase_total,
                                           "Average Total Purchase per Person" : average_age_total_purchase})

# Format the data for cleaner appearance
# Use iloc() to select all the rows for columns that need to be formatted, then applymap() to format the entire selection
age_purchase_summary_df.iloc[ :, 1: ] = age_purchase_summary_df.iloc[ : , 1:].applymap("${:,.2f}".format)

# Add the index name
age_purchase_summary_df.index.name = "Age Ranges"

# Display the summary table
age_purchase_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Ranges,,,,
< 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

    * SN
    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value

* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [23]:
# Group the purchase_data data frame by screen name
top_spenders = purchase_data.groupby("SN")

# Sum each screen name by price to get total purchase value
top_spenders_price = top_spenders["Price"].sum()

# Count the number of purchases per screen name
top_spenders_count = top_spenders["Purchase ID"].count()

# Calculate the average purchase price, use top_spenders_count as the average is dependent on the number of items each person bought
top_spenders_average_price = top_spenders["Price"].mean()

# Create a data frame to display the findings
top_spenders_summary_df = pd.DataFrame({"Purchase Count" : top_spenders_count,
                                        "Average Purchase Price" : top_spenders_average_price,
                                        "Total Purchase Value": top_spenders_price})

# Sort the values of Total Purchase Value in descending order
top_spenders_summary_df = top_spenders_summary_df.sort_values("Total Purchase Value", ascending = False)

# Format the data for cleaner appearance
top_spenders_summary_df.iloc[ : , 1: ] = top_spenders_summary_df.iloc[ : , 1: ].applymap("${:,.2f}".format)

# Display the summary table
top_spenders_summary_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns

* Group by Item ID and Item Name. Perform calculations to obtain the following:

    * Item ID
    * Item Name
    * Purchase Count
    * Item Price
    * Total Purchase Value

* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [24]:
# Group by Item ID and Item Name
item_group = purchase_data.groupby(["Item ID", "Item Name"])

# Calculate the number of times each item was purchased
item_group_count = item_group["SN"].count()

# Use sum() to find the total purchase value of each item as the purchase price varies
item_group_value = item_group["Price"].sum()

# Calculate the item price by averaging the total purchase value
item_group_price = item_group_value / item_group_count

# Create a data frame to display the findings
item_group_summary_df = pd.DataFrame({"Purchase Count" : item_group_count,
                                      "Item Price" : item_group_price,
                                      "Total Purchase Value": item_group_value})

# Sort the values of Purchase Count in descending order
item_group_summary_df = item_group_summary_df.sort_values("Purchase Count", ascending = False)

# Format the data for cleaner appearance
item_group_summary_df.iloc[ : , 1: ] = item_group_summary_df.iloc[ : , 1: ].applymap("${:,.2f}".format)

#Display the summary table
item_group_summary_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

In [25]:
# Create a data frame to display the findings
most_profitable_summary_df = pd.DataFrame({"Purchase Count" : item_group_count,
                                      "Item Price" : item_group_price,
                                      "Total Purchase Value": item_group_value})

# Sort by Purchase Value in descending order
most_profitable_summary_df = most_profitable_summary_df.sort_values("Total Purchase Value", ascending = False)

# Format the data for cleaner appearance
most_profitable_summary_df.iloc[ : , 1: ] = most_profitable_summary_df.iloc[ : , 1: ].applymap("${:,.2f}".format)

#Display the summary table
most_profitable_summary_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
